In [1]:
import math
import os
from tempfile import TemporaryDirectory
from typing import Tuple

import torch
from torch import nn, Tensor
from torch.nn import TransformerEncoder, TransformerEncoderLayer
from torch.utils.data import dataset

import torch.nn.functional as F

#plotting
import matplotlib.pyplot as plt

import data_utils
import models
import importlib
import transformer_models
importlib.reload(data_utils)
importlib.reload(models)
importlib.reload(transformer_models)
from data_utils import *
from models import *
from transformer_models import * 
import torch.optim as optim
from torch.utils.data import DataLoader
import datetime
from tqdm import tqdm 
import pickle
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [2]:
data_dir = "data/"
dataObject = xrdData("data/", device, datasets_avail=['train', 'val', 'test'])

dataObject.make_datasets(1, composition_embedding="compositionseq")
# Create DataLoaders for train and validation sets
train_loader = DataLoader(dataObject.torch_datasets['train'], batch_size=32, shuffle=True)
test_loader = DataLoader(dataObject.torch_datasets['test'], batch_size=32, shuffle=False)  

In [3]:
pickle_path = 'comp_eval_results1.pkl'

# load existing data from pickle file
def load_data(pickle_path):
    try:
        with open(pickle_path, 'rb') as f:
            return pickle.load(f)
    except (EOFError, FileNotFoundError):
        return []  

# save data to a pickle file
def save_data(data, pickle_path):
    with open(pickle_path, 'wb') as f:
        pickle.dump(data, f)


In [4]:
crop_starts_stops = [300, 500, 700, 900, 1100]  
noise_levels = [0.1, 0.2, 0.3, 0.4, 0.5]
drop_freqs = [2, 4, 8, 16, 32] 

log = load_data(pickle_path)

# Training loop with noise level variation
for i in range(5):

    output_dim = 230  # Output dimension
    token_size = 10 #dimension of the tokens 
    TransModel = FullModel(ntoken = output_dim, d_model = token_size, nhead = 10, d_hid=50, nlayers=1, dropout = 0.5).to(device)
    model = TransModel

    weight_decay = 1e-3 
    optimizer = optim.Adam(model.parameters(), lr=0.00001, weight_decay=weight_decay)
    criterion = nn.CrossEntropyLoss()

    max_epochs = 100
    metrics = ["accuracy", "loss"]

    crop_start = crop_starts_stops[i]
    crop_stop = crop_starts_stops[i] 
    noise_level = noise_levels[i]
    drop_freq = drop_freqs[i]
    
    print(f"Run {i+1}: Crop Start/Stop: {crop_start}, Noise Level: {noise_level}, Drop Frequency: {drop_freq}")
    run_data = {'run': i+1, 'parameters': {'crop_start': crop_start, 'noise_level': noise_level, 'drop_freq': drop_freq}, 'epochs': []}
   
    simulator = ExperimentalSimulation(device, crop_start=crop_start, crop_stop=crop_stop, noise_range=noise_level, drop_width=100, drop_freq=drop_freq)
    
    for epoch in range(max_epochs):
        model.train()  
        total_train_loss = 0
        correct_train = 0
        total_train = 0

        for xrd, composition, targets in tqdm(train_loader):
            xrd = simulator.sim(xrd)
            optimizer.zero_grad()
            outputs = model(xrd, composition)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1)
            total_train += targets.size(0)
            correct_train += (predicted == targets).sum().item()

        train_accuracy = 100 * correct_train / total_train
        print(f"Epoch {epoch+1} Training Loss: {total_train_loss}, Training Accuracy: {train_accuracy}")

        run_data['epochs'].append({
        'epoch': epoch,
        'type': 'Train',
        'train_accuracy': train_accuracy,
        'train_loss': total_train_loss
        })

    # Validation 
    model.eval()  # Set the model to evaluation mode
    total_test_loss = 0
    correct_test = 0
    total_test = 0

    with torch.no_grad(): 
        for xrd, composition, targets in tqdm(test_loader):
            xrd = simulator.sim(xrd)
            outputs = model(xrd, composition)
            loss = criterion(outputs, targets)
            total_test_loss += loss.item()

            _, predicted = torch.max(outputs.data, 1) 
            total_test += targets.size(0)
            correct_test += (predicted == targets).sum().item()

    test_accuracy = 100 * correct_test / total_test

    log.append(test_accuracy)

12160
torch.Size([40, 80])
torch.Size([80, 40])
Run 1: Crop Start/Stop: 300, Noise Level: 0.1, Drop Frequency: 2


100%|██████████| 720/720 [00:17<00:00, 40.48it/s]


Epoch 1 Training Loss: 3186.9372828006744, Training Accuracy: 12.593332175724953


100%|██████████| 720/720 [00:16<00:00, 42.86it/s]


Epoch 2 Training Loss: 2689.9953525066376, Training Accuracy: 17.589859350581698


...

Epoch 47 Training Loss: 1298.244311094284, Training Accuracy: 53.67685362042021


100%|██████████| 720/720 [00:26<00:00, 27.31it/s]


Epoch 48 Training Loss: 1286.934745490551, Training Accuracy: 54.37141864907102


 24%|██▍       | 171/720 [00:06<00:20, 27.16it/s]


KeyboardInterrupt: 